In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time

driver = webdriver.Firefox()
base_page = 'https://m******.**'

driver.get(base_page)
driver.maximize_window()

In [ ]:
# скроллинг до конца страницы
SCROLL_PAUSE_TIME = 5

# Get scroll height
last_height = 0

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    
    if new_height == last_height:
        break
    last_height = new_height

In [ ]:
# все заголовки статей на странице
title_list = [item.text for item in driver.find_elements(By.CLASS_NAME, "title")]

# ссылки для всех статей на странице    
href_list=[item.get_attribute('href') for item in driver.find_elements(By.CLASS_NAME, "content-inner > a")]

In [ ]:
df = pd.DataFrame()
df['Название_статьи'] = title_list
df['Ссылка_на_статью'] = href_list
df

In [ ]:
# выгрузка категорий для каждой статьи
idx = 0
for item in driver.find_elements(By.CLASS_NAME, "content-inner"):
    cat_url = []
    cat_title = []
    for element in item.find_elements(By.CLASS_NAME, "article-content-wrap > span.meta-category > a"):
        cat_url.append(element.get_attribute('href'))
        cat_title.append(element.get_attribute('text'))
    count = 1
    for elem in zip(cat_title, cat_url):
        cat_name = str('Категория_') + str(count)
        cat_url = str('Ссылка_на_категорию_') + str(count)
        df.loc[idx,cat_name] = elem[0]
        df.loc[idx,cat_url] = elem[1]
        count+=1
    idx+=1
df.head(3)

In [ ]:
# выгрузка тегов из каждой статьи
idx = 0
date_list = []
for _, raw in df.iterrows():
    driver.get(raw[1])
    tag_name = []
    tag_url = []
    date_list.append(driver.find_elements(By.CLASS_NAME, "meta-date")[0].text)
    
    for item in driver.find_elements(By.CLASS_NAME, "post-tags > a"):
        tag_name.append(item.text)
        tag_url.append(item.get_attribute('href'))
    count = 1
    for elem in zip(tag_name, tag_url):
        tag_n = str('tag_') + str(count)
        tag_u = str('tag_') + str(count) + str('_url')
        df.loc[idx,tag_n] = elem[0]
        df.loc[idx,tag_u] = elem[1]
        count+=1
    idx+=1    
    time.sleep(SCROLL_PAUSE_TIME)
df['Дата'] = date_list
df.head(3)

In [ ]:
cols = list(df)
cols = cols[:2] +[cols[-1]] + cols[2:-1]
df = df[cols]

In [ ]:
with pd.ExcelWriter('m********.xlsx') as writer:
    df.to_excel(writer, sheet_name='Base')

In [ ]:
driver.close()